In [1]:
library(repr)
library(tidyverse)
library(tidymodels)
library(dplyr)
library(RColorBrewer)
options(repr.matrix.max.rows = 10)
set.seed(1234)

options(repr.plot.width = 12, repr.plot.height = 6)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.6
✔ forcats   1.0.1     ✔ stringr   1.6.0
✔ ggplot2   4.0.1     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.2.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.4.1 ──

✔ broom        1.0.11     ✔ rsample      1.3.1 
✔ dials        1.4.2      ✔ tailor       0.1.0 
✔ infer        1.0.9      ✔ tune         2.0.1 
✔ modeldata    1.5.1      ✔ workflows    1.3.0 
✔ parsnip      1.4.0      ✔ workflowsets 1.1.1 
✔ recipes      1.3.1      ✔ yardstick    1.3.2 

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() mask

In [2]:
test_data <- read_csv("https://raw.githubusercontent.com/sophiaymeng/hack-ml/refs/heads/main/test.csv", show_col_types = FALSE)
test_data

city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,⋯,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
<chr>,<dbl>,<dbl>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
iq,2008,27,2008-07-01,0.3048286,0.2336714,0.2489143,0.2558143,0.00,296.5429,⋯,9.40,86.52429,0.00,15.28571,9.442857,NA,NA,NA,20.8,1.3
iq,2008,28,2008-07-08,0.1538500,0.1495857,0.3294429,0.1328857,0.00,296.5457,⋯,7.47,85.65571,0.00,15.08000,9.271429,NA,NA,NA,21.0,0.0
iq,2008,29,2008-07-15,0.2323857,0.2139571,0.2203000,0.1984286,62.84,296.0786,⋯,27.33,93.05571,62.84,16.14286,8.442857,NA,NA,NA,21.0,44.2
iq,2008,30,2008-07-22,0.3270167,0.2091167,0.2837667,0.3095500,52.36,296.5457,⋯,47.59,92.77714,52.36,16.58286,8.600000,26.60,9.775,32.4,20.8,18.1
iq,2008,31,2008-07-29,0.3548000,0.3465500,0.2674167,0.3921167,44.63,297.0443,⋯,39.00,92.04857,44.63,16.87286,9.771429,26.68,11.240,33.4,20.6,36.4
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
sj,2008,13,2008-03-25,0.07785,-0.03990000,0.31047140,0.29624290,27.19,296.9586,⋯,7.55,74.24714,27.19,13.64429,2.885714,25.04286,5.785714,30.0,21.1,1.8
sj,2008,14,2008-04-01,-0.03800,-0.01683333,0.11937140,0.06638571,3.82,298.0814,⋯,3.67,74.60000,3.82,14.66286,2.714286,26.24286,6.814286,30.6,22.2,0.5
sj,2008,15,2008-04-08,-0.15520,-0.05275000,0.13775710,0.14121430,16.96,297.4600,⋯,35.00,75.02714,16.96,14.18429,2.185714,25.00000,5.714286,29.4,21.7,30.7


In [3]:
train_data <- read_csv("https://raw.githubusercontent.com/sophiaymeng/hack-ml/refs/heads/main/test.csv", show_col_types = FALSE)
train_data

city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,⋯,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
<chr>,<dbl>,<dbl>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
iq,2008,27,2008-07-01,0.3048286,0.2336714,0.2489143,0.2558143,0.00,296.5429,⋯,9.40,86.52429,0.00,15.28571,9.442857,NA,NA,NA,20.8,1.3
iq,2008,28,2008-07-08,0.1538500,0.1495857,0.3294429,0.1328857,0.00,296.5457,⋯,7.47,85.65571,0.00,15.08000,9.271429,NA,NA,NA,21.0,0.0
iq,2008,29,2008-07-15,0.2323857,0.2139571,0.2203000,0.1984286,62.84,296.0786,⋯,27.33,93.05571,62.84,16.14286,8.442857,NA,NA,NA,21.0,44.2
iq,2008,30,2008-07-22,0.3270167,0.2091167,0.2837667,0.3095500,52.36,296.5457,⋯,47.59,92.77714,52.36,16.58286,8.600000,26.60,9.775,32.4,20.8,18.1
iq,2008,31,2008-07-29,0.3548000,0.3465500,0.2674167,0.3921167,44.63,297.0443,⋯,39.00,92.04857,44.63,16.87286,9.771429,26.68,11.240,33.4,20.6,36.4
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
sj,2008,13,2008-03-25,0.07785,-0.03990000,0.31047140,0.29624290,27.19,296.9586,⋯,7.55,74.24714,27.19,13.64429,2.885714,25.04286,5.785714,30.0,21.1,1.8
sj,2008,14,2008-04-01,-0.03800,-0.01683333,0.11937140,0.06638571,3.82,298.0814,⋯,3.67,74.60000,3.82,14.66286,2.714286,26.24286,6.814286,30.6,22.2,0.5
sj,2008,15,2008-04-08,-0.15520,-0.05275000,0.13775710,0.14121430,16.96,297.4600,⋯,35.00,75.02714,16.96,14.18429,2.185714,25.00000,5.714286,29.4,21.7,30.7
